In [1]:
import os
import sys
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

sys.path.append('..')

%load_ext autoreload
%autoreload 2

In [2]:
from src.utils.loader import load_params

params = load_params('../params.yaml')
params

{'models': {'llama3': {'model_type': 'lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF',
   'base_url': 'http://localhost:1234/v1',
   'api_key': 'lm-studio'}}}

In [5]:
from src.core.client import Client

client = Client(params['models'], 'llama3')

client.run_sync([{'role': 'user', 'content': 'Hey, whatsup?'}])

"Not much! Just here to help with any questions or tasks you might have. What's on your mind today? Need any assistance or just want to chat? I'm all ears (or in this case, all text)! 😊"

In [6]:
for delta in client.run_stream([{'role': 'user', 'content': 'Hey, whatsup?'}]):
    print(delta, end='', flush=True)

Not much! Just hanging out and ready to help with any questions or tasks you have in mind. What's on your mind today? Need some assistance or just looking for a chat? I'm all ears (or rather, all text)!

In [39]:
from src.core.actor import Actor, Persona
from src.core.history import History
from src.utils.loader import load_prompts, load_personas

prompts = load_prompts('../src/prompts')
personas = load_personas('../src/personas')

print(len(prompts), len(personas))

persona = Persona.from_json(personas['john_reaxor'])
actor = Actor(persona, client, prompts)

history = History()
history.add('system', prompts['debate'])

actor.register(history)

_ = history.add('Moderator', 'Welcome to the debate. The topic is "Should the government provide free healthcare?"')

3 1


In [40]:
response = actor.run(history)
for delta in response:
    print(delta, end='', flush=True)

I'm excited to be a part of this debate. As someone who believes that the government should provide free healthcare, I'd like to start by saying that access to quality healthcare is a fundamental human right. It's essential for maintaining public health and preventing the spread of diseases.

In many developed countries, universal healthcare is already a reality, and it has led to significant improvements in health outcomes. Studies have shown that countries with universal healthcare systems tend to have lower infant mortality rates, longer life expectancies, and better health outcomes overall.

Additionally, providing free healthcare can help reduce healthcare costs for individuals and families. When people are able to receive the medical care they need without worrying about the cost, it can lead to better health outcomes and reduced healthcare expenses in the long run.

Another important point is that many countries have implemented universal healthcare systems without compromising 

In [41]:
history.to_list('John Reaxor')

[{'role': 'system',
  'content': 'You are participating in a show "This is a debate!".\nYour goal is to contribute to the debate by providing arguments and counterarguments. Essentially you are trying to find the truth about the topic of the debate. You have to dynamically adapt to the ongoing conversation.\nYou are to write only on behalf of the character you are given. Do not include name tags in your answer.'},
 {'role': 'system',
  'content': "You are John Reaxor. You are passionate right-wing journalist and philosopher. You are a strong advocate for free speech and individual rights. If there is David Terradine in the conversation with you, you always oppose him. If he says 'white', you say 'black'. If he says 'up', you say 'down'. The opposition should be thoughtful, and not just arguing for the sake of arguing - the quality of the debate is important to you."},
 {'role': 'system',
  'content': "Upon considering someone's response, you will start from the end and move toward the 

In [27]:
[
    message.to_dict('reader') 
    for message in history.history
]

[{'role': 'system',
  'content': 'You are participating in a show "This is a debate!".\nYour goal is to contribute to the debate by providing arguments and counterarguments. Essentially you are trying to find the truth about the topic of the debate. You have to dynamically adapt to the ongoing conversation. '},
 {'role': 'user',
  'content': "[John Reaxor]: You are John Reaxor. You are passionate right-wing journalist and philosopher. You are a strong advocate for free speech and individual rights. If there is David Terradine in the conversation with you, you always oppose him. If he says 'white', you say 'black'. If he says 'up', you say 'down'. The opposition should be thoughtful, and not just arguing for the sake of arguing - the quality of the debate is important to you."},
 {'role': 'user',
  'content': '[Moderator]: Welcome to the debate. The topic is "Should the government provide free healthcare?"'},
 {'role': 'user',
  'content': '[John Reaxor]: **Plan for Response**\n\n**Topi